## Multiple Contracts

To demonstrate how we can use multiple contracts, we will create a strategy that looks at overnight price gaps for 2 contracts
It chooses the name that has the largest percentage overnight gap and goes long that name. The strategy exits by the end of the day
If price moves against us, we use a stop loss to get out. We have 15 minute bars of Coke (KO) and Pepsi (PEP) stock prices

In [62]:
%%checkall
import pandas as pd
import math
import pyqstrat as pq
import numpy as np
from collections import defaultdict
from dataclasses import dataclass
from types import SimpleNamespace
from typing import Sequence


FILENAMES = {'ko': './support/coke_15_min_prices.csv.gz', 'pep': './support/pepsi_15_min_prices.csv.gz'}


@dataclass
class OvernightReturn:
    name: str
    on_ret: float
    timestamps: np.ndarray
    prices: np.ndarray
    
    
def create_online_returns(contracts: list[str]) -> dict[np.datetime64, OvernightReturn]:
    on_rets: dict[np.datetime64, OvernightReturn] = {}
    for name in contract_names:
        prices = pd.read_csv(FILENAMES[name], usecols=['date', 'c'])
        prices.columns = ['timestamp', 'c']
        prices['date'] = prices.timestamp.values.astype('M8[D]')
        prices['on_ret'] = np.where(prices.date > prices.date.shift(1), prices.c / prices.c.shift(1) - 1, np.nan)
        date_rets = prices[np.isfinite(prices.on_ret) & (prices.on_ret > 0)]
        dates = date_rets.date.values.astype('M8[D]')
        on_ret = date_rets.on_ret.values
        for i, date in enumerate(dates):
            if date not in on_rets or on_ret[i] > on_rets[date].on_ret:
                date_prices = prices[prices.date == date]
                on_rets[date] = OvernightReturn(name, on_ret[i], date_prices.timestamp.values, date_prices.c.values)
    return on_rets
    
    
# Lets create an entry rule. We can use pq.FiniteRiskEntryRule as a template and modify it so
# that it trades the correct contract based on the data in on_rets
@dataclass
class MultipleContractEntryRule:
    '''
    A rule that generates orders with stops. Modified version of pq.FiniteRiskEntrRule
    Args:
        reason_code: Reason for the orders created used for display
        entry_contracst: A dict containing contract name we want to enter for each date
        price_func: A function that returns price given a contract and timestamp
        long: Whether we want to go long or short
        percent_of_equity: How much to risk per trade as a percentage of current equity.
            Used to calculate order qty so that if we get stopped out, we don't lose 
            more than this amount. Of course if price gaps up or down rather than moving smoothly,
            we may lose more.
        stop_price_ind: 
    '''
    reason_code: str
    entry_contracts: dict[np.datetime64, str]
    price_func: pq.PriceFunctionType
    long: bool = True
    percent_of_equity: float = 0.1
    stop_price_ind: str | None = None
    min_price_diff: float = 0
    single_entry_per_day: bool = False
        
    def __call__(self,
                 contract_group: pq.ContractGroup,
                 i: int,
                 timestamps: np.ndarray,
                 indicator_values: SimpleNamespace,
                 signal_values: SimpleNamespace,
                 account: pq.Account,
                 current_orders: Sequence[pq.Order],
                 strategy_context: pq.StrategyContextType) -> list[pq.Order]:
        
        date = timestamps[i].astype('M8[D]')
        entry_contract: str | None = self.entry_contracts.get(date)
        if entry_contract is None: return []
        if contract_group.name != entry_contract: return []
            
        timestamp = timestamps[i]
        if self.single_entry_per_day:
            date = timestamp.astype('M8[D]')
            trades = account.get_trades_for_date(contract_group.name, date)
            if len(trades): return []

        contract = contract_group.get_contract(contract_group.name)
        entry_price_est = self.price_func(contract, timestamps, i, strategy_context)
        if math.isnan(entry_price_est): return []
        
        if self.stop_price_ind:
            _stop_price_ind = getattr(indicator_values, self.stop_price_ind)
            stop_price = _stop_price_ind[i]
        else:
            stop_price = 0.

        if self.long and (entry_price_est - stop_price) < self.min_price_diff: return []
        if not self.long and (stop_price - entry_price_est) < self.min_price_diff: return []
        
        curr_equity = account.equity(timestamp)
        risk_amount = self.percent_of_equity * curr_equity
        _order_qty = risk_amount / (entry_price_est - stop_price)
        order_qty = math.floor(_order_qty) if _order_qty > 0 else math.ceil(_order_qty)
        if math.isclose(order_qty, 0.): return []
        order = pq.MarketOrder(contract=contract, 
                               timestamp=timestamp, 
                               qty=order_qty,
                               reason_code=self.reason_code)
        return [order]


def create_price_dataframe(on_rets: dict[np.datetime64, OvernightReturn]) -> pd.DataFrame:
    dfs: list[pd.DataFrame] = []
    for date, on_ret in on_rets.items():
        df = pd.DataFrame({'timestamp': on_ret.timestamps, 'price': on_ret.prices})
        df['date'] = df.timestamp.values.astype('M8[D]')
        df['contract'] = on_ret.name
        df['stop_price'] = df.groupby('date').price.first()
        dfs.append(df)
    prices = pd.concat(dfs)
    return prices
    

def create_price_function(on_rets: dict[np.datetime64, OvernightReturn]) -> pq.PriceFunctionType:
    price_dict: dict[str, dict[np.datetime64, float]] = defaultdict(dict)
    for on_ret in on_rets.values():
        price_dict[on_ret.name].update({on_ret.timestamps[i]: on_ret.prices[i] for i in range(len(on_ret.timestamps))})
    price_function = pq.PriceFuncDict(price_dict=price_dict)
    return price_function


if __name__ == '__main__':
    contract_names = ['ko', 'pep']

    on_rets: dict[np.datetime64, OvernightReturn] = create_online_returns(contract_names)
    prices = create_price_dataframe(on_rets)

    strat_builder = pq.StrategyBuilder(data=prices)
    for contract in contract_names:
        strat_builder.add_contract(contract)
    # add the stop price so we can refer to it in
    strat_builder.add_indicator('stop_price', prices.stop_price.values) 
    price_function = create_price_function(on_rets)
    strat_builder.set_price_function(price_function)

    # FiniteRiskEntryRule allows us to enter trades and get out with a limited loss when a stop is hit.
    # This enters market orders, if you want to use limit orders, set the limit_increment argument
    entry_contracts = {date: on_ret.name for date, on_ret in on_rets.items()}
    entry_rule = MultipleContractEntryRule(
        reason_code='POS_OVERNIGHT_RETURN',  # this is useful to know why we entered a trade
        entry_contracts=entry_contracts,  # contract to enter for each date
        price_func=price_function, 
        long=True,  # whether we enter a long or short position
        percent_of_equity=0.1,  # set the position size so that if the stop is hit, we lose no more than this
        # stop price is used for position sizing.  Also, we will not enter if the price is already below 
        # stop price for long trades and vice versa
        stop_price_ind='stop_price',
        single_entry_per_day=True)  # if we are stopped out, do we allow re-entry later in the day

    # ClosePositionExitRule fully exits a position using either a market or limit order
    # In this case, we want to exit at EOD so we are flat overnight
    exit_rule_stop = pq.ClosePositionExitRule(   
        reason_code='STOPPED_OUT',
        price_func=price_function)

    # Exit when the stop price is reached
    exit_rule_eod = pq.ClosePositionExitRule(
        reason_code='EOD',
        price_func=price_function)

    # Setup the rules we setup above so they are only called when the columns below in our data dataframe are true
    strat_builder.add_series_rule('overnight_ret_positive', entry_rule, position_filter='zero')
    strat_builder.add_series_rule('eod', exit_rule_eod, position_filter='positive')
    strat_builder.add_series_rule('stop', exit_rule_stop, position_filter='positive')

    # create the strategy and run it
    strategy = strat_builder()
    strategy.run()


running typecheck
Success: no issues found in 1 source file

running flake8
flake8 success


PQException: overnight_ret_positive not found in data: Index(['timestamp', 'price', 'date', 'contract', 'stop_price'], dtype='object')

In [65]:
prices[np.isfinite(prices.stop_price)]

,timestamp,price,date,contract,stop_price
